In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv('matches.csv', index_col=0)

print(matches.shape)

(2244, 28)


In [4]:
matches['Team'].value_counts()

Team
Chelsea                     113
Aston Villa                 113
Tottenham Hotspur           113
Manchester United           113
Newcastle United            112
Manchester City             112
Arsenal                     112
Liverpool                   112
Brentford                   112
Nottingham Forest           112
Fulham                      112
Brighton and Hove Albion    112
Crystal Palace              112
Everton                     112
Wolverhampton Wanderers     112
Bournemouth                 112
West Ham United             112
Leicester City               74
Southampton                  74
Luton Town                   38
Sheffield United             38
Burnley                      38
Leeds United                 38
Ipswich Town                 36
Name: count, dtype: int64

In [5]:
matches.dtypes

Date              object
Time              object
Comp              object
Round             object
Day               object
Venue             object
Result            object
GF                 int64
GA                 int64
Opponent          object
xG               float64
xGA              float64
Poss               int64
Attendance       float64
Captain           object
Formation         object
Opp Formation     object
Referee           object
Match Report      object
Notes            float64
Sh                 int64
SoT                int64
Dist             float64
FK                 int64
PK                 int64
PKatt              int64
Season             int64
Team              object
dtype: object